In [1]:
from MTGSqliteDatabase import MTGSqliteDatabase
from Index import Index
import pickle
import logging
from pprint import pprint
import sqlite3
from sqlite3 import Error

logging.basicConfig(level=logging.INFO)

In [2]:
db:MTGSqliteDatabase = MTGSqliteDatabase()
db.Update()

INFO:root:Database found in ./Sqlite/AllPrintings.sqlite. I'll use the available version.


Connection to SQLite DB successful


INFO:root:Server sha256: 19d3b6389b899d6f79455cb11c3161c0467a682570a581b3fce222d6f14f3ce4
INFO:root:Local sha256:  19d3b6389b899d6f79455cb11c3161c0467a682570a581b3fce222d6f14f3ce4
INFO:root:Available version is Uptodate


## Function related to database

In [3]:
def execute_read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as e:
        print(f"The error '{e}' occurred")
        
def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection


connection = create_connection("./Sqlite/AllPrintings.sqlite")

Connection to SQLite DB successful
Done


## Pickle the English data

In [ ]:
cursor = connection.cursor()
result = None
try:    
    cursor.execute("SELECT id, flavorText, originalText, text, name, flavorName, types, artist \
                        FROM cards WHERE availability LIKE '%paper%' ")
except Error as e:
        print(f"The error '{e}' occurred")

index = Index()

for row in cursor:
    all_text = ""
    for element in row[1:]:
        if element != None:
            all_text += " "+str(element)
    
    keywords = index.standardize_keywords(all_text, ADDING_CARD=True)

    index.add(row[0], keywords)

with open("pickle/mtg_db_en_sqlite.pkl", "wb") as f:
    pickle.dump(index, f)
    
print("Done")

In [4]:
query = ['butt', 'kick']
ranking = index.ranked_search(query)
ranking = sorted(ranking, key=ranking.get, reverse=True)
#print("Number of results:", len(ranking), ranking)
print( execute_read_query(connection, "SELECT id, artist, availability, flavorText, name \
                                            FROM cards WHERE id="+str(ranking[0]) ) )

The error 'unrecognized token: "94db"' occurred
None


## Pickle the Italian data

In [7]:
cursor = connection.cursor()
result = None

query = "SELECT cards.id, cards.artist, foreign_data.flavorText, foreign_data.name, foreign_data.text, foreign_data.type \
            FROM cards LEFT JOIN foreign_data ON cards.uuid = foreign_data.uuid \
                WHERE cards.availability LIKE '%paper%' AND foreign_data.language = \"Italian\""
try:
    cursor.execute(query)
except Error as e:
        print(f"The error '{e}' occurred")

index = Index()

for row in cursor:
    all_text = ""
    for element in row[1:]:
        if element != None:
            all_text += " "+str(element)
    
    keywords = index.standardize_keywords(all_text, ADDING_CARD=True)

    index.add(row[0], keywords)

with open("pickle/mtg_db_it_sqlite.pkl", "wb") as f:
    pickle.dump(index, f)
    
print("Done")

Done


In [8]:
query = ['foresta', 'palude']
ranking = index.ranked_search(query)
ranking = sorted(ranking, key=ranking.get, reverse=True)
print( execute_read_query(connection, "SELECT id, artist, availability, flavorText, name \
                                            FROM cards WHERE id="+str(ranking[0]) ) )

[(41211, 'Ron Spencer', 'mtgo,paper', None, 'Last Stand')]
